<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/parity_classification_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies (run once)
!pip install pennylane numpy scikit-learn

# Imports
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp
from sklearn.model_selection import KFold
from sklearn.svm import SVC

# Data generation
def generate_parity_data(n_bits):
    patterns = np.array(
        [list(map(int, np.binary_repr(i, width=n_bits)))
         for i in range(2**n_bits)]
    )
    labels = np.array([1 if p.sum() % 2 == 0 else -1 for p in patterns])
    return patterns, labels

# 1) Analytic CNOT cascade
def analytic_cnot(n_bits):
    dev = qml.device("default.qubit", wires=n_bits + 1)
    @qml.qnode(dev)
    def circuit(x):
        for i, bit in enumerate(x):
            qml.RX(np.pi * bit, wires=i)
        for i in range(n_bits):
            qml.CNOT(wires=[i, n_bits])
        return qml.expval(qml.PauliZ(n_bits))

    X, Y = generate_parity_data(n_bits)
    preds = np.array([circuit(x) for x in X])
    acc = (preds == Y).mean()
    print(f"Analytic CNOT       Accuracy: {acc:.3f}")

# 2) Variational QNN + k-Fold CV
def variational_qnn(n_bits, n_layers=3, epochs=50, k_folds=5, lr=0.2):
    dev = qml.device("default.qubit", wires=n_bits)
    @qml.qnode(dev, interface="autograd")
    def circuit(params, x):
        for i, bit in enumerate(x):
            qml.RX(np.pi * bit, wires=i)
        for layer in range(n_layers):
            for i in range(n_bits):
                qml.RY(params[layer][i], wires=i)
            for i in range(n_bits - 1):
                qml.CNOT(wires=[i, i + 1])
        return qml.expval(qml.PauliZ(0))

    def hinge_loss(y_true, y_pred):
        return pnp.mean(pnp.maximum(0, 1 - y_true * y_pred))

    X, Y = generate_parity_data(n_bits)
    kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    fold_acc = []

    for train_idx, test_idx in kf.split(X):
        params = pnp.random.randn(n_layers, n_bits, requires_grad=True) * 0.1
        opt = qml.GradientDescentOptimizer(stepsize=lr)
        for _ in range(epochs):
            def cost(p):
                preds = pnp.array([circuit(p, x) for x in X[train_idx]])
                return hinge_loss(Y[train_idx], preds)
            params = opt.step(cost, params)

        preds_test = pnp.array([circuit(params, x) for x in X[test_idx]])
        acc = (pnp.sign(preds_test) == Y[test_idx]).mean()
        fold_acc.append(acc)

    print(f"Variational QNN     CV Accuracy: {np.mean(fold_acc):.3f} ± {np.std(fold_acc):.3f}")

# 3) Quantum Kernel + SVM
def quantum_kernel_svm(n_bits):
    dev = qml.device("default.qubit", wires=n_bits)
    @qml.qnode(dev)
    def feature_map(x, y=None):
        for i, bit in enumerate(x):
            qml.RY(np.pi * bit, wires=i)
        for i in range(n_bits):
            for j in range(i + 1, n_bits):
                qml.CNOT(wires=[i, j])
        if y is not None:
            for i, bit in enumerate(y):
                qml.RY(-np.pi * bit, wires=i)
        return qml.expval(qml.PauliZ(0))

    X, Y = generate_parity_data(n_bits)
    N = len(X)
    gram = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            gram[i, j] = feature_map(X[i], X[j])

    svc = SVC(kernel="precomputed")
    svc.fit(gram, Y)
    preds = svc.predict(gram)
    acc = (preds == Y).mean()
    print(f"Quantum Kernel + SVM Accuracy: {acc:.3f}")

# Run all three methods on 5 bits
N_BITS = 5
analytic_cnot(N_BITS)
variational_qnn(N_BITS)
quantum_kernel_svm(N_BITS)